In [4]:
# from vpi_bot_skills import AuthenChatUser

# auth = AuthenChatUser()
# auth.get_auth('18uedfklashdfkjh')

d:\SOFTWARE\Python310\env\msbot\lib\site-packages\botbuilder\schema\__init__.py:80: UserWarning: The Bot Framework Python SDK is being retired with final long-term support ending in November 2023, after which this repository will be archived. There will be no further feature development, with only critical security and bug fixes within this repository being undertaken. Existing bots built with this SDK will continue to function. For all new bot development we recommend that you adopt Power Virtual Agents.
  warn(


<coroutine object AuthenChatUser.get_auth at 0x000001B713505A80>

from vpi_bot_skills.chat_database_skills import ChatWithDataBase

ChatWithDataBase().chat_database()

In [13]:
import json
x = '{"type":"question","value":"công thức đạo hàm hàm mũ"}'
z = json.loads(x)
type(z)

dict

In [2]:
from vpi_teams_bot import VpiSkKernel2
kernel = VpiSkKernel2()
skcontext = kernel.get_context('test')

table_name = kernel.get_semantic_skill('databaseSkills')['genSqlQuery']

schemas = """
Table: students
Collumns:
	- id: id 
	- name: student's name
	- birthday: student's birthday
	- address: adress

Table: grades
Collumns:
	- id : id
	- studentid: student's id in students table
	- courseid: course's id in cousrse table
	- score: score of stundet id learning this course id

Table: courses
Collumns:
	- id: id
	- courseName: name of course 
		- courseName value: ['Toán','Vật Lý','Hóa']
	- lecturer: name of teacher teach this course
"""

skcontext['data_schemas'] = schemas
skcontext['data_objective'] = 'tìm số lượng sinh viên có điểm môn toán cao hơn 90'
# skcontext['data_objective'] = 'tìm tên giảng viên dậy môn lý'
# skcontext['data_objective'] = 'Lấy tên các học sinh có điểm toán > 80'
# skcontext['data_objective'] = 'Tìm số sinh viên học tất cả các môn'

await table_name.invoke_async(context=skcontext)
print(skcontext.result)



Answer:
SELECT COUNT(*) 
FROM students s 
INNER JOIN grades g 
ON s.id = g.studentid 
INNER JOIN courses c 
ON g.courseid = c.id 
WHERE c.courseName = 'Toán' 
AND g.score > 90;


In [10]:
sql = skcontext.result.strip()[skcontext.result.find(':'):] if 'Answer' in skcontext.result else skcontext.result
print(sql)
check = skcontext.result.strip()
out = check[check.find(':') + 1:] if 'Answer' in check else check
print(out)

SELECT COUNT(*) 
FROM students s 
INNER JOIN grades g 
ON s.id = g.studentid 
INNER JOIN courses c 
ON g.courseid = c.id 
WHERE c.courseName = 'Toán' 
AND g.score > 90;

SELECT COUNT(*) 
FROM students s 
INNER JOIN grades g 
ON s.id = g.studentid 
INNER JOIN courses c 
ON g.courseid = c.id 
WHERE c.courseName = 'Toán' 
AND g.score > 90;


In [4]:
import re

# Sử dụng regex và re.sub() để loại bỏ các ký tự đặc biệt
prcessed_sql = re.sub(r'\W+', ' ', sql)

print(prcessed_sql)

 SELECT COUNT DISTINCT s id FROM students s INNER JOIN grades g ON s id g studentid INNER JOIN courses c ON g courseid c id WHERE c courseName IN Toán Vật Lý Hóa 


In [11]:
from sqlalchemy import create_engine
import pandas as pd

connector = create_engine('mysql://root:''@localhost/student').connect()

df = pd.read_sql_table('students', connector)
df.head()

,ID,Name,Birthday,Address
0,1,Trang,2000-01-01,Hồ Chí Minh
1,2,Quỳnh,1999-05-10,Cần Thơ
2,3,Long,2002-09-12,Hà Nội
3,4,Lâm,2001-08-02,Bắc Giang
4,5,Hằng,2002-12-12,Hưng Yên


In [12]:
from sqlalchemy.sql import text

# sql_text = text('select * from students')
sql_text = text(sql)
# sql_text = text(prcessed_sql)
connector.execute(sql_text).fetchall()

[(1,)]

In [18]:
df2 = pd.read_sql(sql_text, connector)
df2.head()

,lecturer


In [11]:
# chat = kernel.get_semantic_skill('chatSkills')
chat = kernel.import_semantic_skill_from_directory('./vpi_bot_skills/','chatSkills')

In [2]:
# func1 = kernel.import_semantic_skill_from_directory('./vpi_bot_skills','databaseSkills')['getTableName']

In [3]:
# sql_context = kernel.get_context('sql')
# sql_context['schema'] = str(ChatWithDataBase.SCHEMAS) 
# sql_context.put('tìm danh sách những nhân viên có địa điểm làm việc ở Hồ Chí Minh')

# print(func1(sql_context))

In [4]:
import pandas as pd

df = pd.read_csv('./data/diemthi2019.csv')
df.head()

,Unnamed: 0,Dia,GDCD,Hoa,Li,Ma_mon_ngoai_ngu,Ngoai_ngu,Sinh,Su,Toan,Van,sbd
0,0,7.75,8.75,NaN,NaN,N1,7.6,NaN,8.25,8.2,8.00,24008611
1,1,NaN,NaN,4.5,8.25,N1,8.0,6.0,NaN,8.6,6.17,51000032
2,2,4.25,5.75,NaN,NaN,NaN,NaN,NaN,4.00,5.2,4.75,51000005
3,3,4.75,5.00,NaN,NaN,NaN,NaN,NaN,2.25,4.4,4.50,51000021
4,4,8.25,7.50,NaN,NaN,N1,4.4,NaN,7.75,5.8,6.00,51000013


In [5]:
df_head = df.head().to_csv(index=False)
print(df_head)

Unnamed: 0,Dia,GDCD,Hoa,Li,Ma_mon_ngoai_ngu,Ngoai_ngu,Sinh,Su,Toan,Van,sbd
0,7.75,8.75,,,N1,7.6,,8.25,8.2,8.0,24008611
1,,,4.5,8.25,N1,8.0,6.0,,8.6,6.17,51000032
2,4.25,5.75,,,,,,4.0,5.2,4.75,51000005
3,4.75,5.0,,,,,,2.25,4.4,4.5,51000021
4,8.25,7.5,,,N1,4.4,,7.75,5.8,6.0,51000013



In [6]:
from pandasai import PandasAI
from pandasai.llm import OpenAI
import os

In [10]:
llm = OpenAI(os.getenv(key='OPENAI_API_KEY'))
pd_ai = PandasAI(llm, enable_cache= False)
kq = pd_ai(df, 'tìm tìm 10 sbd có điểm văn cao nhất')
print(kq)

--- Logging error ---
Traceback (most recent call last):
  File "D:\SOFTWARE\Python310\lib\logging\__init__.py", line 1103, in emit
    stream.write(msg + self.terminator)
  File "D:\SOFTWARE\Python310\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\u0111' in position 55: character maps to <undefined>
Call stack:
  File "D:\SOFTWARE\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\SOFTWARE\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\SOFTWARE\Python310\env\msbot\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "d:\SOFTWARE\Python310\env\msbot\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "d:\SOFTWARE\Python310\env\msbot\lib\site-packages\ipykerne

149017    29029394
210203    55009659
210340    55009794
210353    55009803
223611    34009156
325959    37000035
366362    19007241
550972    34006800
645266    29007662
690032    29023280
Name: sbd, dtype: int64


In [9]:
func2 = kernel.get_semantic_skill('databaseSkills')['getPythonCode']

In [11]:
print(pd_ai.last_code_generated)

None


In [13]:
sql_context2 = kernel.get_context('sql2')
question = "tìm 9 sbd có điểm toán cao nhất"
sql_context2['input'] = question
sql_context2['origin_question'] = question
sql_context2['num_rows'] = str(df.shape[0])
sql_context2['num_cols'] = str(df.shape[1])
sql_context2['df_head'] = df_head
	
print(func2.invoke(context= sql_context2))



<startCode>
# Get the top 9 SBDs with the highest Toan scores
df.sort_values(by='Toan', ascending=False).head(9)['sbd']
<endCode>


In [36]:
pd_ai.last_code_generated
pd_ai.run_code(pd_ai.last_code_generated, df)

136269    46000056
211674    22008906
212034    22009272
260813    19013622
346036     1068360
Name: sbd, dtype: int64

In [ ]:
top_10_sbd_toan_cao_nhat = df.nlargest(10, 'Toan')['sbd']
top_10_sbd_toan_cao_nhat